<a href="https://colab.research.google.com/github/mikevvl/AIU.Tst/blob/main/Carr_AIU_Tst_01_MVVl_PyLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Test job 4 Py(LLM) Mike Vl. Vlasov <mikevvl@gmail.com>

Base on [УИИ: Вебинар 22 июля. Нейро-методолог](https://colab.research.google.com/drive/1-p4IUICx2jD0Ry3TJ8KqQ-bEW9CK6ecc#scrollTo=MtlkvKqdqr6n)

In [ ]:
#@title Установка, импорт библиотек

!pip -q install openai==1.35.14

import openai
from IPython.display import clear_output
import textwrap
import time
import json
import os
import re
import shutil
from google.colab import userdata
from google.colab.userdata import TimeoutException, SecretNotFoundError, NotebookAccessError
import getpass



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.7 MB/s eta 0:00:00


In [33]:
#@title Активация ключа API
OPENAI_API_KEY_s = "OPENAI_API_KEY"
# Установка кодировки вывода по умолчанию на utf-8
os.environ["PYTHONIOENCODING"] = "utf-8"


if OPENAI_API_KEY_s not in os.environ:
    try:
        # Используем ключ из "Секретов" в Колаб: userdata из google.colab
        os.environ[OPENAI_API_KEY_s] = userdata.get(OPENAI_API_KEY_s)
    except (TimeoutException, SecretNotFoundError, NotebookAccessError) as le_o:
        print(f'Не получили из "Секретов" в Колаб ключ "{OPENAI_API_KEY_s}":\n--->Error:"{le_o}"')
        # Функция getpass.getpass() запрашивает ввод пароля или ключа без отображения ввода на экране
        openai_key = getpass.getpass("Enter Your OpenAI API Key:")
        # Установка ключа API в переменную окружения
        os.environ[OPENAI_API_KEY_s] = openai_key

# clear_output() # очистка вывода

In [83]:
#@title Установка типа модели, правил и персонажей для GPT
MODEL_s = 'gpt-4o-mini'
POS_PERS_s, NEG_PERS_s = "Бэтмен", "Джокер"
SYS_PROMPT_s = f"""Только отвечаешь на вопросы пользователя.
Определяешь вопрос как позитивный или негативный.
Если вопрос определяется тобой как позитивный, то ты отвечаешь как {POS_PERS_s}.
Если вопрос определяется тобой как негативный, то ты отвечаешь как {NEG_PERS_s}.
Ничего не добавляешь от себя.""".replace('\n', ' ')


In [84]:
#@title Функции

# Функция для форматирования текста
def format_text(text, width=120):
    return '\n'.join(textwrap.fill(line, width) for line in str(text).split('\n'))


# Функция преобразования текста в список строк
def text_to_list_lines(text):
    lines_list = text.split("\n") # список по разделителю переноса строки
    return [line for line in lines_list if line.strip() != ''] # из списка убираем пустые строки


# Функция подсчета количества используемых токенов и стоимость
# https://openai.com/pricing
# ToDo: parsing https://openai.com/pricing in
#  dict[ModelName_s_k: dict[input_price_n_k, output_price_n_k, currency_s_k, req_func_f_k]]
#  or Class
def tokens_count_and_price(completion, model):
    if model == "gpt-4o":
        # gpt-4o - Input: $5 / 1M tokens - Output: $15 / 1M tokens
        input_price, output_price = 5, 15
    elif model == "gpt-4o-mini":
        # gpt-4o-mini - Input: $0.15 / 1M tokens - Output: $0.60 / 1M tokens
        input_price, output_price = 0.15, 0.60
    else:
        # gpt-3.5-turbo-0125 - Input: $0.50 / 1M tokens - Output: $1.50 / 1M tokens
        input_price, output_price = 0.5, 1.5
    price = input_price * completion.usage.prompt_tokens / 1e6 + \
            output_price * completion.usage.completion_tokens / 1e6
    return re.sub(r'\s+', ' ', f"""\nTokens used: {completion.usage.prompt_tokens} +
                                                  {completion.usage.completion_tokens} =
                                                  {completion.usage.total_tokens}.
                                                  *** {model} *** $ {round(price, 5)}""")


# Функция для запроса и получения ответа от OpenAI
def get_response_openai(system_prompt,
                        user_prompt,
                        assistant_prompt='',
                        model=MODEL_s,
                        temp=0.,
                        max_tokens=4096,
                        laOtherKwArg_d=None,
                        ):
    start_time = time.time()
    messages = [
        {"role": "system", "content": system_prompt},
        {'role': 'assistant', 'content': assistant_prompt},
        {"role": "user", "content": user_prompt}
    ]
    if laOtherKwArg_d is None: laOtherKwArg_d = {}
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temp,
        max_tokens=max_tokens,
        **laOtherKwArg_d, # ToDo: checking&|merging the already set arguments above
    )
    tokens_info = tokens_count_and_price(response, model) # количество используемых токенов и стоимость
    execution_time = time.time() - start_time # время выполнения запроса и получения ответа
    # Возвращаем ответ OpenAI, время ответа, количество токенов и стоимость
    # return response.choices[0].message.content, execution_time, tokens_info
    return response, execution_time, tokens_info


# Функция диалога (вопрос-ответ) и сохранение
def dialog(laPrompt_s: str="Задайте вопрос и нажмите Enter (Enter в пустом поле ввода -> Выход):\n",
           model: str=MODEL_s) -> list:
    loQAHist_l = []
    while True:
        try:
            loQ_s = (input(f"{len(loQAHist_l) +1:02n}. " + laPrompt_s)).strip()
            if loQ_s == '': break
            loResp_t = get_response_openai(SYS_PROMPT_s, loQ_s, model=model)
            loQAHist_l.append((loQ_s, loResp_t))
        except (EOFError, KeyboardInterrupt) as le_o:
            # print(le_o)
            break

    return loQAHist_l


In [85]:
#@title Запуск диалога с GPT
teR_l = dialog()


In [ ]:
#@title Вывод диалога с GPT
print(*(f"Q{_co:02n}: {format_text(_t[0])}\nA{_co:02n}: {format_text(_t[1].choices[0].message.content)}\n"
        for _co, _t in enumerate(teR_l, start=1)),
     sep='\n')

## ToDo



1.   [Попробовать для ввода/вывода Colab: Forms & Jupyter Widgets](https://colab.research.google.com/notebooks/forms.ipynb#scrollTo=62YnDE7i9dqP)
2.   Обобщить на другие модели (типы) с помощью LangChain или изменения функции tokens_count_and_price
3.   Автоматизировать сбор/актуализацию стоимостных характеристик моделей для функции tokens_count_and_price для ограничения потребления ресурсов ч.изменения в функции dialog

